# Interacting with a bitcoind node using Python
This guide assumes you have a bitcoind node running and available at `localhost:8332`. If not you should refer to this repository's `README.md` for how to set that up.

Requires:
```
python-bitcoinrpc (1.0)
```

In [1]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
import json

Let's set the RPC user and password so that we can instantiate a connection to bitcoind:

In [4]:
rpc_user = 'rpcuser'
rpc_password = 'rpcsecret'

Now we can go ahead and connect to the bitcoind node:

In [12]:
rpc_conn = AuthServiceProxy("http://%s:%s@localhost:8332"%(rpc_user, rpc_password), timeout=10000)

Let's test the connection:

In [13]:
rpc_conn.getinfo()

{'blocks': 195148,
 'connections': 8,
 'deprecation-warning': 'WARNING: getinfo is deprecated and will be fully removed in 0.16. Projects should transition to using getblockchaininfo, getnetworkinfo, and getwalletinfo before upgrading to 0.16',
 'difficulty': Decimal('2190865.970102859'),
 'errors': '',
 'paytxfee': Decimal('0E-8'),
 'protocolversion': 70015,
 'proxy': '',
 'relayfee': Decimal('0.00001000'),
 'testnet': False,
 'timeoffset': 0,
 'version': 150100}

Note that the latest block at the time of writing was `498016`. You'll notice that if you just started up your bitcoind node it probably has far fewer blocks than that. This is because it is still in the process of downloading the full blockchain and getting up to speed. 

One thing to consider is that only blocks found on disk can be accessed using the RPC connection. So if you want to inspect a block you can, just ensure you are passing the hash of an early block. For example:

In [134]:
rpc_conn.getblock('00000000000005cbac4729863520767efc50c505fe2743d5d8abb016683388e3')

{'bits': '1a05db8b',
 'chainwork': '00000000000000000000000000000000000000000000001b9bfea5cbeb9a9fc4',
 'confirmations': 132,
 'difficulty': Decimal('2864140.507810974'),
 'hash': '00000000000005cbac4729863520767efc50c505fe2743d5d8abb016683388e3',
 'height': 201286,
 'mediantime': 1349034457,
 'merkleroot': 'b680578ad66388b672782b3092aef4cbc74e45daf2c3a4152c42bd2cb31f2b61',
 'nextblockhash': '0000000000000341a9c8a4f050fe3273edd0a602ecb9b94377c5c1c54185358a',
 'nonce': 338505311,
 'previousblockhash': '000000000000014125fedb75009ba81c07ac3e4c1481ffb9a4f9ee02d4178a7a',
 'size': 86877,
 'strippedsize': 86877,
 'time': 1349035927,
 'tx': ['ac304fa1978ec6bae8acfca077348e5d92040dd67733b4b593fdddb116d2504b',
  '48adf12b094bafe7dca55653862c72c69a1fa049bccd1fc397da46e0bb0e69a2',
  '2acbef7e5a866d0134841e61aeeae4b60f043834ac5b389d94af3cdbea1d6d77',
  '1d26080b54021589ef33f3482254ef5f06a3ec0c8ea3c4d37b4ec994dfb88618',
  '9d6f4eb5fba393408981fa9835f141aed9e18b51a9f3ef32446e69d423dbd11a',
  '76a766

Now we can define some helper functions to get us additional information. We will always pass the RPC connection (`rpc_conn`) to each function to remind us that we need a live connection in case it has timed out.

In [15]:
def get_block_hash(rpc_conn, index):
    return rpc_conn.getblockhash(index)

def get_block(rpc_conn, hash):
    return rpc_conn.getblock(hash)

def get_raw_transaction(rpc_conn, tx):
    out = rpc_conn.getrawtransaction(tx, 1)
    return out

def get_tx_outputs(tx):
    outputs = [float(i['value']) for i in tx['vout']]
    return outputs

Now we can get a block using the block height as opposed to the block's raw hash:

In [135]:
block = get_block(rpc_conn,get_block_hash(rpc_conn, 201286))

In [136]:
block

{'bits': '1a05db8b',
 'chainwork': '00000000000000000000000000000000000000000000001b9bfea5cbeb9a9fc4',
 'confirmations': 138,
 'difficulty': Decimal('2864140.507810974'),
 'hash': '00000000000005cbac4729863520767efc50c505fe2743d5d8abb016683388e3',
 'height': 201286,
 'mediantime': 1349034457,
 'merkleroot': 'b680578ad66388b672782b3092aef4cbc74e45daf2c3a4152c42bd2cb31f2b61',
 'nextblockhash': '0000000000000341a9c8a4f050fe3273edd0a602ecb9b94377c5c1c54185358a',
 'nonce': 338505311,
 'previousblockhash': '000000000000014125fedb75009ba81c07ac3e4c1481ffb9a4f9ee02d4178a7a',
 'size': 86877,
 'strippedsize': 86877,
 'time': 1349035927,
 'tx': ['ac304fa1978ec6bae8acfca077348e5d92040dd67733b4b593fdddb116d2504b',
  '48adf12b094bafe7dca55653862c72c69a1fa049bccd1fc397da46e0bb0e69a2',
  '2acbef7e5a866d0134841e61aeeae4b60f043834ac5b389d94af3cdbea1d6d77',
  '1d26080b54021589ef33f3482254ef5f06a3ec0c8ea3c4d37b4ec994dfb88618',
  '9d6f4eb5fba393408981fa9835f141aed9e18b51a9f3ef32446e69d423dbd11a',
  '76a766

The block is a dictionary which has a key called `tx` which lists each transaction included in the block. We can grab one of these transactions and inspect it further:

In [142]:
sample_tx_hash = block['tx'][1]
tx = get_raw_transaction(rpc_conn, sample_tx_hash)
tx

{'blockhash': '00000000000005cbac4729863520767efc50c505fe2743d5d8abb016683388e3',
 'blocktime': 1349035927,
 'confirmations': 216,
 'hash': '48adf12b094bafe7dca55653862c72c69a1fa049bccd1fc397da46e0bb0e69a2',
 'hex': '01000000038f09b018a8fc9736b116eb8b63824a8eb95d1142df9d97406a7442043726d43b010000008a47304402200b5946f3f3cf843fb71718e748d40ec739e8650f5070ad82c11471b5856d089f02201c208b88e8e706354e6abaef231974c1da345760e2404887cad2903ccf4ab96c014104186d879b63467176196f34e34b65174d74a88171fcc8146565352b82759ee223a705820d0d60702192e21f16ec37480c6015ec469a8f63c1fb17358fdeffb86dffffffff055885e7b65f394328f88cee25fdbe3c5013d4bdcc05d9ac17a0e529a20d8182010000008b483045022100cd580aa577f7106ab9cb6da43422eda8cc2402ae02cc503482497300d94b02b702201092208479c2015b65d7949c1c5d58182ae0e6493210a3288603009dabe70883014104f9f92ea030627b8735ae5c9a93a8cbbbab57fe95b8f0cd80fb6f9e7ae49d04e2a611a1ad5cf37885c587f9eb590bbf042aeb4a6a38138b264df84fb0d70ee7f3ffffffff7d34bd78f7519de6c2b857ea82a148b1a1bbe6275ef8e65f4dcf995

Here we have the raw transaction. This contains all the information for this transaction including where the transaction originated, where it went, and its corresponding amounts.

By looking at the difference between the output from a transaction and its inputs we can determine the actual mining fees paid by the transactor:

In [143]:
def get_tx_fee(rpc, tx):
    tx_inputs = 0
    tx_outputs = 0
    for vin in tx['vin']:
        tx_id = vin['txid']
        ind = vin['vout']
        inner_tx = get_raw_transaction(rpc, tx_id)
        inner_val = inner_tx['vout'][ind]['value']
        tx_inputs += inner_val
    for vout in tx['vout']:
        tx_outputs += vout['value']
    return tx_inputs - tx_outputs

In [144]:
get_tx_fee(rpc_conn, tx)

JSONRPCException: -5: No such mempool transaction. Use -txindex to enable blockchain transaction queries. Use gettransaction for wallet transactions.

In [27]:
block = get_block(rpc_conn,rpc_conn.getbestblockhash())

We can also batch process blocks:

In [51]:
commands = [ [ "getblockhash", height] for height in range(10) ]
block_hashes = rpc_conn.batch_(commands)
blocks = rpc_conn.batch_([ [ "getblock", h ] for h in block_hashes ])
block_times = [ block["time"] for block in blocks ]

This returns the epoch times for each block:

In [52]:
block_times

[1231006505,
 1231469665,
 1231469744,
 1231470173,
 1231470988,
 1231471428,
 1231471789,
 1231472369,
 1231472743,
 1231473279]